# GPkit calculated variables

The issue was with how we were declaring `lambda` functions in a loop that they were returning the same variable.

__The Fix__
- declare the `lambda` with a variable which has a different default to force the scope of the loop control variable to be local to the lambda function

_Example_
- when declaring a lambda function to make a calculated, fixed variable where a variable `var` is the loop control variable, do the following:

```python
lambda m, V=var : m[V]
```

### Imports

In [2]:
from gpkit import Model, Variable

In [3]:
import numpy as np

Test to see if we can create models with calculated values

## Test a Model with dependant derived variables

In [4]:
class Product(Model):
    def setup(self, rate, cv):
        # create a rate variable
        self.lam = Variable('\\lambda', rate)
        self.cv = Variable('c^2', cv)

$$ 

In [5]:
class Cell(Model):
    def setup(self, products):
        
        # calulate total rate 
        self.lam = Variable('\\bar{\\lambda}', 
                            lambda c : np.sum([c[p.lam] for p in products]))
        
        # calulate the x (as fraction of total rate)
        self.x = [Variable('x_%i'%i, lambda c : c[p.lam]/c[self.lam]) for i,p in enumerate(products)]
        
        # some measure of cv
        self.cv = Variable('cv^2')
        
        # set up some constraints
        constraints = [self.x[i]*self.lam >= p.lam for i,p in enumerate(products)]
        constraints.append(1 >= np.sum(self.x))
        constraints.append(self.cv >= np.sum([self.x[i]*p.cv for i,p in enumerate(products)]))
        constraints.extend(products)  # have to return the products too
        
        return constraints

In [6]:
# create products
products = [Product(4, 1), Product(6, 0.5)]

In [7]:
# create cell
cell = Cell(products)

In [8]:
# minimize the cost (sum of x)
cell.cost = cell.cv

In [9]:
# try and solve
cell.solve()

Couldn't auto-differentiate linked variable Cell.x_0
  (to raise the error directly for debugging purposes, set gpkit.settings["ad_errors_raise"] to True)
Couldn't auto-differentiate linked variable Cell.x_1
  (to raise the error directly for debugging purposes, set gpkit.settings["ad_errors_raise"] to True)


ValueError: The constraint Cell.x_0 + Cell.x_1 <= 1 is infeasible by 20.000000%

_Failure:_ Everything is substitutied so there are no free variables to optimize

### Try and formulate without using dependant variables 

_Calculate $\mathbf{x}$ only_

In [82]:
class Cell2(Model):
    def setup(self, products):
                        
        # calulate the x (as fraction of total rate)
        self.x = [
            Variable('x_%i'%i, 
                     lambda c : c[p.lam]/np.sum([c[pp.lam] for pp in products]))  # cant divide int by posynomial
            for i,p in enumerate(products)]
        
        # some measure of cv
        self.cv = Variable('cv^2')
        
        # set up some constraints
        constraints = []
#         constraints.append(self.x[i]*self.lam >= p.lam for i,p in enumerate(products))
#         constraints.append(1 >= np.sum(self.x))
        constraints.append(self.cv >= np.sum([self.x[i]*p.cv for i,p in enumerate(products)]))
        constraints.extend(products)  # have to return the products too
        
        return constraints

In [83]:
# create products
products = [Product(4, 1), Product(6, 0.5)]

In [84]:
# create cell
cell2 = Cell2(products)

In [85]:
# minimize the cost (sum of x)
cell2.cost = cell2.cv

In [86]:
print(cell2.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.0112 seconds.

Cost
----
 0.9

Free Variables
--------------
     | Cell21
cv^2 : 0.9

Constants
---------
        | Cell21
    x_0 : 0.6
    x_1 : 0.6

        | Product4
\lambda : 4
    c^2 : 1

        | Product5
\lambda : 6
    c^2 : 0.5

Sensitivities
-------------
        | Product4
    c^2 : +0.67
\lambda : -0.4

        | Product5
\lambda : +0.4
    c^2 : +0.33

Tightest Constraints
--------------------
       | Cell21
    +1 : cv^2 >= x_0*Product4.c^2 + x_1*Product5.c^2



In [14]:
.4+.6*.5

0.7

In [15]:
# try and solve
print(cell2.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.0111 seconds.

Cost
----
 0.9

Free Variables
--------------
     | Cell2
cv^2 : 0.9

Constants
---------
        | Cell2
    x_0 : 0.6
    x_1 : 0.6

        | Product2
\lambda : 4
    c^2 : 1

        | Product3
\lambda : 6
    c^2 : 0.5

Sensitivities
-------------
        | Product2
    c^2 : +0.67
\lambda : -0.4

        | Product3
\lambda : +0.4
    c^2 : +0.33

Tightest Constraints
--------------------
       | Cell2
    +1 : cv^2 >= x_0*Product2.c^2 + x_1*Product3.c^2



the sensitivities for the rates also seem off. Increasing the rate of the first product, with higher CV should increase the cost function (raise the cv). Increasing the rate of the lower-variability product should decrease the overall variation.

### Repeat example but with manual control of all variables

In [16]:
a1 = Variable('a_1', 6)
b1 = Variable('b_1', 1)

a2 = Variable('a_2', 4)
b2 = Variable('b_2', 0.5)

# declare x which is a derived variable
abar = Variable('\\bar{a}', lambda m : m[a1] + m[a2])
x1 = Variable('x_1', lambda m : m[a1]/(m[a1]+m[a2]))
x2 = Variable('x_2', lambda m : m[a2]/(m[a1]+m[a2]))

# result
c = Variable('c')

# constraints
constraints = [
    c >= b1*x1 + b2*x2,
    1 >= x1 + x2,
    abar >= a1 + a2,
]

m = Model(c, constraints)

In [17]:
print(m.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.0182 seconds.

Cost
----
 0.8

Free Variables
--------------
c : 0.8

Constants
---------
\bar{a} : 10
    a_1 : 6
    a_2 : 4
    b_1 : 1
    b_2 : 0.5
    x_1 : 0.6
    x_2 : 0.4

Sensitivities
-------------
b_1 : +0.75
b_2 : +0.25
a_1 : +0.15
a_2 : -0.15

Tightest Constraints
--------------------
    +1 : c >= b_1*x_1 + b_2*x_2



This works!

_Now try to use the calculated `abar` in calculations for `x`_

### Use calculated variable in calculated variable

In [18]:
a1 = Variable('a_1', 6)
b1 = Variable('b_1', 1)

a2 = Variable('a_2', 4)
b2 = Variable('b_2', 0.5)

# declare x which is a derived variable
abar = Variable('\\bar{a}', lambda m : m[a1] + m[a2])
x1 = Variable('x_1', lambda m : m[a1]/m[abar])
x2 = Variable('x_2', lambda m : m[a2]/m[abar])

# result
c = Variable('c')

# constraints
constraints = [
    c >= b1*x1 + b2*x2,
    1 >= x1 + x2,
    abar >= a1 + a2,
]

m = Model(c, constraints)

In [19]:
print(m.solve().table())

Couldn't auto-differentiate linked variable x_2
  (to raise the error directly for debugging purposes, set gpkit.settings["ad_errors_raise"] to True)


KeyError: \bar{a}

This doesn't work. Have to calculate things individually

### Test with list comprehension

The gpkit documentation mentions that we may have to use `ad.admath` instead of `numpy` functions.

In [56]:
a = [Variable('a_0', 6), Variable('a_1', 4)]
b = [Variable('b_0', 1), Variable('b_1', 0.5)]

c = Variable('c')
abar = Variable('\\bar{a}', lambda m : np.sum([m[var] for var in a]))

x_1 = Variable('x_1',
               lambda m : m[a[0]]/np.sum([m[var] for var in a])
              )

x_2 = Variable('x_2',
               lambda m : m[a[1]]/np.sum([m[var] for var in a])
              )

# x = [Variable('x_%i'%i, 
#               lambda m : m[a[i]]/np.sum([m[var] for var in a])
#              ) for i,_ in enumerate(a)]
# trying to create x with list comprehension breaks it

x = [x_1, x_2]

# create x in a loop instead of list comprehension
# x = []
# for i, avar in enumerate(a):
#     v = Variable('x_%i'%i,
#                 lambda m : m[avar]/np.sum([m[var] for var in a]))
#     x.append(v)

constraints = [
#     c >= x_1*b[0] + x_2*b[1],
    c >= x[0]*b[0] + x[1]*b[1],
#     c >= np.dot(x,b),  # this over constrians
#     1 >= x_1 + x_2,  # aparently this constraint is superfluous...
    abar >= np.sum(a)
]

m = Model(c, constraints)

In [57]:
print(m.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.0285 seconds.

Cost
----
 0.8

Free Variables
--------------
c : 0.8

Constants
---------
\bar{a} : 10
    a_0 : 6
    a_1 : 4
    b_0 : 1
    b_1 : 0.5
    x_1 : 0.6
    x_2 : 0.4

Sensitivities
-------------
b_0 : +0.75
b_1 : +0.25
a_0 : +0.15
a_1 : -0.15

Tightest Constraints
--------------------
    +1 : c >= x_1*b_0 + x_2*b_1



_ITERATIONS_
- [x] np list comp for `abar` sub
- [x] np list comp for `abar` constr
- [x] np list comp for `x1` & `x2` sub
  - [ ] make list comp a function
- [x] dot prod for `c` constraint
- [ ] list comp for `x` vector
  - creating `x` with list comprehension breaks it. It seems there is an assignment issue when using calculated variables

#### ISSUE: Creating Variables with List Comprehension has strange effect

In [58]:
vals = [1, 4, 5]
v = [Variable('v_%i'%i, val) for i,val in enumerate(vals)]
tot = Variable('total')
m3 = Model(tot, [tot >= np.sum(v)])

In [60]:
print(m3.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.0109 seconds.

Cost
----
 10

Free Variables
--------------
total : 10

Constants
---------
v_0 : 1
v_1 : 4
v_2 : 5

Sensitivities
-------------
v_2 : +0.5
v_1 : +0.4
v_0 : +0.1

Tightest Constraints
--------------------
    +1 : total >= v_0 + v_1 + v_2



#### TEST: adding attributes to a class
__Result:__ can add arbitrary attributes to a class

In [62]:
class Sample:
    name = 'bill'

In [63]:
test = Sample()

In [64]:
test.name

'bill'

In [65]:
# try assigning an attribute that was not there
test.x = 0.5

In [66]:
test.x

0.5

- no problem with just adding attributes willy-nilly
- on each product.cell could attach an `x` attribute that is calculated

#### TEST: different methods of accessing the variables

In [69]:
a = [Variable('a_0', 6), Variable('a_1', 4)]
b = [Variable('b_0', 1), Variable('b_1', 0.5)]

c = Variable('c')
abar = Variable('\\bar{a}', lambda m : np.sum([m[var] for var in a]))

x_1 = Variable('x_1',
               lambda m : m[a[0]]/np.sum([m[var] for var in a])
              )

x_2 = Variable('x_2',
               lambda m : m[a[1]]/np.sum([m[var] for var in a])
              )

# x = [Variable('x_%i'%i, 
#               lambda m : m[a[i]]/np.sum([m[var] for var in a])
#              ) for i,_ in enumerate(a)]
# trying to create x with list comprehension breaks it

# x = [x_1, x_2]

# create x in a loop instead of list comprehension
x = []
for i, avar in enumerate(a):
    v = Variable('x_%i'%i,
                lambda m : m[avar]/np.sum([m[var] for var in a]))
    x.append(v)

constraints = [
#     c >= x_1*b[0] + x_2*b[1],
    c >= x[0]*b[0] + x[1]*b[1],
#     c >= np.dot(x,b),  # this over constrians
#     1 >= x_1 + x_2,  # aparently this constraint is superfluous...
    abar >= np.sum(a)
]

m = Model(c, constraints)

In [70]:
print(m.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.016 seconds.

Cost
----
 0.6

Free Variables
--------------
c : 0.6

Constants
---------
\bar{a} : 10
    a_0 : 6
    a_1 : 4
    b_0 : 1
    b_1 : 0.5
    x_0 : 0.4
    x_1 : 0.4

Sensitivities
-------------
b_0 : +0.67
a_0 : -0.6
a_1 : +0.6
b_1 : +0.33

Tightest Constraints
--------------------
    +1 : c >= x_0*b_0 + x_1*b_1



### try using other objects

In [206]:
class Prod(Model):
    def setup(self, rate, cv):
        self.a = Variable('a', rate)
        self.b = Variable('b', cv)
        self.x = Variable('x')

In [207]:
products = [Prod(6, 1), Prod(4, 0.5)]

In [213]:
# calculated variables
abar = Variable('\\bar{a}', 
                lambda m : np.sum(m[p.a] for p in products))

# this gives the same failure
# i = 0
# products[i].x = Variable('x_%i'%i,
#                          lambda m : m[products[i].a]/np.sum(m[p.a] for p in products))

# i = 1
# products[i].x = Variable('x_%i'%i,
#                          lambda m : m[products[i].a]/np.sum(m[p.a] for p in products))

# create the vector of x
x = []
for i, p in enumerate(products):
    v = Variable('x_%i'%i,
                lambda m, P=p : m[P.a]/np.sum([m[pp.a] for pp in products]))
    x.append(v)
    
# free variable
c = Variable('c')

In [214]:
# constraints
constraints = []
# constraints.append(c >= np.sum([p.b*p.x for p in products]))
constraints.append(c >= np.dot(x, [p.b for p in products]))
constraints.append(abar >= np.sum([p.a for p in products]))
constraints.extend(products)

m = Model(c, constraints)

In [216]:
print(m.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.00652 seconds.

Cost
----
 0.8

Free Variables
--------------
c : 0.8

Constants
---------
\bar{a} : 10
    x_0 : 0.6
    x_1 : 0.4

        | Prod22
      a : 6
      b : 1

        | Prod23
      a : 4
      b : 0.5

Sensitivities
-------------
  | Prod22
b : +0.75
a : +0.15

  | Prod23
b : +0.25
a : -0.15

Tightest Constraints
--------------------
    +1 : c >= x_0*Prod22.b + x_1*Prod23.b



/home/snill/aa-refactor/aa-py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  This is separate from the ipykernel package so we can avoid doing imports until


__THIS SOLVES THE ISSUE__

#### Details on working the lambda function issue

In [191]:
# # try updating substituions
# ## THIS APPROACH WORKS
# m.substitutions[products[0].x] = (
#     lambda y : y[products[0].a]/np.sum([y[p.a] for p in products])
# )

# m.substitutions[products[1].x] = (
#     lambda y : y[products[1].a]/np.sum([y[p.a] for p in products])
# )

In [192]:
# # try and update subs with a loop
# ## THIS APPROACH FAILS
# m.substitutions.update({
#     p.x : lambda y=i : y[p.a]/np.sum([y[pp.a] for pp in products]) for i,p in enumerate(products)
# })

In [210]:
# ## THIS WORKS

# for p in products:
#     m.substitutions[p.x] = lambda y, pl=p: y[pl.a]/np.sum([y[pp.a] for pp in products])

There is an issue with using lambda functions in a loop or with list comprehension

One potential solution is to use a default value for the function as discussed here: https://stackoverflow.com/questions/33983980/lambda-in-for-loop-only-takes-last-value

See also: https://stackoverflow.com/questions/139819/why-results-of-map-and-list-comprehension-are-different

- the generator function is not local to the lambda

Finally, consider this from the official python documentation: https://docs.python.org/3/faq/programming.html#why-do-lambdas-defined-in-a-loop-with-different-values-all-return-the-same-result

In [184]:
# the lambda function will be evaluated to the last state of p
# therefore, if we change p, the evaluation of the lambda function will be different
p = products[0]

## Test $\bar{\lambda}$ as a free variable

May want to assume $\lambda_i$ is NOT an input. This would be the case if the feeder quanitites are free variables (such would affect the production rate at these cells).

In [8]:
from gpkit import Variable, Model
import numpy as np

In [9]:
class Prod(Model):
    def setup(self, rate, cv):
        self.a = Variable('a', rate)
        self.b = Variable('b', cv)
        self.x = Variable('x')

In [5]:
products = [Prod(6, 1), Prod(4, 0.5)]

In [11]:
# calculated variables
# abar = Variable('\\bar{a}', 
#                 lambda m : np.sum(m[p.a] for p in products))

abar = Variable('\\bar{a}')
x = []
for i, p in enumerate(products):
    v = Variable('x_%i'%i,
                lambda m, P=p : m[P.a]/np.sum([m[pp.a] for pp in products]))
    x.append(v)
    
# free variable
c = Variable('c')

In [12]:
# constraints
constraints = []
# constraints.append(c >= np.sum([p.b*p.x for p in products]))
constraints.append(c >= np.dot(x, [p.b for p in products]))
constraints.append(abar >= np.sum([p.a for p in products]))
constraints.extend(products)

m = Model(c**2*abar, constraints)

In [14]:
print(m.solve().table())

Using solver 'cvxopt'
Solving for 2 variables.
Solving took 0.0101 seconds.

Cost
----
 6.4

Free Variables
--------------
\bar{a} : 10
      c : 0.8

Constants
---------
x_0 : 0.6
x_1 : 0.4

    | Prod
  a : 6
  b : 1

    | Prod1
  a : 4
  b : 0.5

Sensitivities
-------------
  | Prod
b : +1.5
a : +0.9

  | Prod1
b : +0.5
a : +0.1

Tightest Constraints
--------------------
    +2 : c >= x_0*Prod.b + x_1*Prod1.b
    +1 : \bar{a} >= Prod.a + Prod1.a



/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)




## Second Attempt

In [26]:
pfail = Variable('\\rho_{fail}', 0.1)    # failure parts
pgood = Variable('\\rho_{good}', lambda c : 1-c[pfail])  # good parts
lamin = Variable('\\lambda_{in}', 100)
lamout = Variable('\\lambda_{out}')

m = Model(1/lamout, [pgood*lamin >= lamout])

In [27]:
m

<gpkit.Model object containing 1 top-level constraint(s) and 3 variable(s)>

In [29]:
m.solve()

Couldn't auto-differentiate linked variable \rho_{good}
  (to raise the error directly for debugging purposes, set gpkit.settings["ad_errors_raise"] to True)


KeyError: \rho_{fail}

_Failure:_ the source for the calculated variables also has to be constrained in the model